In [26]:
# import sys
# !{sys.executable} -m pip install protobuf==3.20.3

# import sys
# !{sys.executable} -m pip install tf2onnx

# import sys
# !{sys.executable} -m pip install onnxruntime



In [27]:
import tensorflow as tf
import tf2onnx
import onnx
import onnxruntime as ort

In [28]:
model = tf.keras.models.load_model("Crop.h5")

In [29]:
model

In [31]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 64)        36928     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 6, 6, 64)         0

In [32]:
input_signature = [
    tf.TensorSpec(
        shape=[None, 64, 64, 3], 
        dtype=tf.float32,
        name="input"
    )
]


In [33]:
onnx_model, _ = tf2onnx.convert.from_keras(
    model,
    input_signature=input_signature,
    opset=13,
    output_path="crop_disease_model.onnx"
)


In [34]:
import onnx

onnx_model = onnx.load("crop_disease_model.onnx")
onnx.checker.check_model(onnx_model)

print("ONNX model is valid ✅")


ONNX model is valid ✅


In [35]:
import onnxruntime as ort
import numpy as np

session = ort.InferenceSession("crop_disease_model.onnx")

input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name

dummy_input = np.random.rand(1, 64, 64, 3).astype(np.float32)

output = session.run([output_name], {input_name: dummy_input})
print("Output shape:", output[0].shape)


Output shape: (1, 15)


In [36]:
# Class name -> index
class_indices = {
    "Pepper__bell___Bacterial_spot": 0,
    "Pepper__bell___healthy": 1,
    "Potato___Early_blight": 2,
    "Potato___Late_blight": 3,
    "Potato___healthy": 4,
    "Tomato_Bacterial_spot": 5,
    "Tomato_Early_blight": 6,
    "Tomato_Late_blight": 7,
    "Tomato_Leaf_Mold": 8,
    "Tomato_Septoria_leaf_spot": 9,
    "Tomato_Spider_mites_Two_spotted_spider_mite": 10,
    "Tomato__Target_Spot": 11,
    "Tomato__Tomato_YellowLeaf__Curl_Virus": 12,
    "Tomato__Tomato_mosaic_virus": 13,
    "Tomato_healthy": 14
}


In [37]:
idx_to_class = {v: k for k, v in class_indices.items()}


In [38]:
from tensorflow.keras.preprocessing import image

def predict_with_onnx(image_path, session, class_indices):
    img = image.load_img(image_path, target_size=(64, 64))
    img = image.img_to_array(img) / 255.0
    img = np.expand_dims(img, axis=0).astype(np.float32)

    input_name = session.get_inputs()[0].name
    output_name = session.get_outputs()[0].name

    preds = session.run([output_name], {input_name: img})[0]

    idx_to_class = {v: k for k, v in class_indices.items()}
    class_id = np.argmax(preds)
    confidence = np.max(preds)

    return idx_to_class[class_id], confidence


In [39]:
label, conf = predict_with_onnx(
    "PlantVillage/test/Pepper__bell___Bacterial_spot/c9e13908-bce6-4ad7-831f-a4a408cabf72___JR_B.Spot 3198.JPG",
    session,
    class_indices
)

print(label, conf)


Pepper__bell___Bacterial_spot 1.0


In [40]:
label, conf = predict_with_onnx(
    "PlantVillage/test/Potato___healthy/111cd9d8-4d27-4225-be9e-d29b21cf56b9___RS_HL 5419.JPG",
    session,
    class_indices
)

print(label, conf)


Potato___healthy 0.99987924
